In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from torch.optim import lr_scheduler

from sklearn.metrics import r2_score
from sklearn import preprocessing
#import scipy.io as sio

from models import VAE,AEBase,Predictor
from models import DNN
import numpy as np
import pandas as pd
import models
import utils as ut
import copy

from scipy import stats


In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.jpeg_quality rcparam was deprecated in M

In [2]:
from scipy.stats import pearsonr

# Parameters

In [3]:
# Define parameters
epochs = 500 #200,500,1000  
#dim_au_in = 20049
dim_au_out = 512 #8, 16, 32, 64, 128, 256,512
dim_dnn_in = dim_au_out
dim_dnn_out=1
select_drug = 'Entinostat'
na = 1

# Import data

In [4]:
data_r=pd.read_csv('data/GDSC2_expression.csv',index_col=0)
label_r=pd.read_csv('data/GDSC2_label_9drugs.csv',index_col=0)

In [5]:
label_r=label_r.fillna(na)

In [6]:
hvg,adata = ut.highly_variable_genes(data_r)

In [7]:
selected_idx = label_r.loc[:,select_drug]!=na

In [8]:
data_r.columns = adata.var_names

In [9]:
hvg.sum()

5116

# Your is gene-cell, mine is cell-gene

In [10]:
#data = data_r.loc[selected_idx,:]
data = data_r.loc[selected_idx,hvg]

In [11]:
label = label_r.loc[selected_idx,select_drug]
#sscaler = preprocessing.StandardScaler(with_mean=True, with_std=True)
mmscaler = preprocessing.MinMaxScaler()
lbscaler = preprocessing.MinMaxScaler()

data = mmscaler.fit_transform(data)
label = lbscaler.fit_transform(label.values.reshape(-1,1))
#label = label.values.reshape(-1,1)

In [12]:
print(np.std(data))
print(np.mean(data))

0.25270809454700544
0.24264587007274258


In [13]:
data.mean(axis=0)


array([0.4758466 , 0.0788965 , 0.20600574, ..., 0.12230791, 0.0555018 ,
       0.40267655])

In [14]:
print(data.max())
print(data.min())

1.0000000000000002
0.0


In [15]:
data.shape

(732, 5116)

In [16]:
label_r.shape

(804, 9)

# Split test train

In [17]:
from sklearn.model_selection import train_test_split
X_train_all, X_test, Y_train_all, Y_test = train_test_split(data, label, test_size=0.2, random_state=42)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train_all, Y_train_all, test_size=0.2, random_state=42)

In [18]:
print(data.shape)
print(label.shape)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(732, 5116)
(732, 1)
(468, 5116) (468, 1)
(147, 5116) (147, 1)


In [19]:
print(X_train.max())
print(X_train.min())

1.0000000000000002
0.0


# AE MODEL

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)
torch.cuda.set_device(device)

cuda:0


# Add all data to AE

In [21]:
X_trainTensor = torch.FloatTensor(X_train).to(device)
X_validTensor = torch.FloatTensor(X_valid).to(device)
X_testTensor = torch.FloatTensor(X_test).to(device)
X_allTensor = torch.FloatTensor(data).to(device)
#X_alltrainTensor = torch.FloatTensor(X_train_all).to(device)


Y_trainTensor = torch.FloatTensor(Y_train).to(device)
Y_validTensor = torch.FloatTensor(Y_valid).to(device)

# construct TensorDataset
train_dataset = TensorDataset(X_trainTensor, X_trainTensor)
valid_dataset = TensorDataset(X_validTensor, X_validTensor)
test_dataset = TensorDataset(X_testTensor, X_testTensor)
all_dataset = TensorDataset(X_allTensor, X_allTensor)

X_trainDataLoader = DataLoader(dataset=train_dataset, batch_size=200, shuffle=True)
X_validDataLoader = DataLoader(dataset=valid_dataset, batch_size=200, shuffle=True)
X_allDataLoader = DataLoader(dataset=all_dataset, batch_size=200, shuffle=True)

In [22]:
dataloader = X_trainDataLoader

In [23]:
#dataloaders_train = {'train':X_trainDataLoader,'val':X_validDataLoader}

In [24]:
X_trainDataLoader.dataset.tensors[0].shape[0]

468

# The model

In [25]:
model = models.PretrainedPredictor(input_dim=5116,latent_dim=512,hidden_dims=[2048,1024], 
                            hidden_dims_predictor=[256,128],drop_out_predictor=0.6,
                            pretrained_weights='saved/models/GDSCnew_ae.pkl',freezed=False)

In [26]:
print(model)

PretrainedPredictor(
  (encoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=5116, out_features=2048, bias=True)
      (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.3, inplace=False)
    )
    (1): Sequential(
      (0): Linear(in_features=2048, out_features=1024, bias=True)
      (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.3, inplace=False)
    )
  )
  (bottleneck): Linear(in_features=1024, out_features=512, bias=True)
  (predictor): Predictor(
    (predictor): Sequential(
      (0): Sequential(
        (0): Linear(in_features=512, out_features=256, bias=True)
        (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Dropout(p=0.6, inplace=False)
      )
      (1): Sequential(
        (0): Linear(in_features=256, out_features=128, bias=True)
        (1): BatchNorm1d(128, eps=1e-05, momentu

In [27]:
#model = VAE(dim_au_in=data_r.shape[1],dim_au_out=128)
if torch.cuda.is_available():
    model.cuda()
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-2)
loss_function = nn.MSELoss()

# Decay LR by a factor of 0.1 every 7 epochs
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer)

In [28]:
# Load data
# data type conversion

# y_trainTensor = torch.FloatTensor(Y_train).to(device)
# y_validTensor = torch.FloatTensor(Y_valid).to(device)

# construct TensorDataset
trainreducedDataset = TensorDataset(X_trainTensor, Y_trainTensor)
validreducedDataset = TensorDataset(X_validTensor, Y_validTensor)

trainDataLoader_p = DataLoader(dataset=trainreducedDataset, batch_size=200, shuffle=True)
validDataLoader_p = DataLoader(dataset=validreducedDataset, batch_size=200, shuffle=True)

In [29]:
dataloaders_train = {'train':trainDataLoader_p,'val':validDataLoader_p}

In [30]:
model,report = ut.train_predictor_model(model,dataloaders_train,
                                        optimizer,loss_function,epochs,exp_lr_scheduler,save_path="saved/models/pre_pre_model.pkl")

Epoch 0/499
----------
train Loss: 0.00082221. Learning rate = 0.01
val Loss: 0.00148126. Learning rate = 0.01
Epoch 1/499
----------
train Loss: 0.00050854. Learning rate = 0.01
val Loss: 0.00113465. Learning rate = 0.01
Epoch 2/499
----------
train Loss: 0.00046358. Learning rate = 0.01
val Loss: 0.00092783. Learning rate = 0.01
Epoch 3/499
----------
train Loss: 0.00039383. Learning rate = 0.01
val Loss: 0.00075124. Learning rate = 0.01
Epoch 4/499
----------
train Loss: 0.00035015. Learning rate = 0.01
val Loss: 0.00071736. Learning rate = 0.01
Epoch 5/499
----------
train Loss: 0.00028305. Learning rate = 0.01
val Loss: 0.00062340. Learning rate = 0.01
Epoch 6/499
----------
train Loss: 0.00029385. Learning rate = 0.01
val Loss: 0.00043134. Learning rate = 0.01
Epoch 7/499
----------
train Loss: 0.00027323. Learning rate = 0.01
val Loss: 0.00026184. Learning rate = 0.01
Epoch 8/499
----------
train Loss: 0.00027987. Learning rate = 0.01
val Loss: 0.00037578. Learning rate = 0.01
E

val Loss: 0.00025432. Learning rate = 0.001
Epoch 74/499
----------
train Loss: 0.00001206. Learning rate = 0.001
val Loss: 0.00026041. Learning rate = 0.001
Epoch 75/499
----------
train Loss: 0.00001190. Learning rate = 0.001
val Loss: 0.00026524. Learning rate = 0.001
Epoch 76/499
----------
train Loss: 0.00001167. Learning rate = 0.001
val Loss: 0.00027210. Learning rate = 0.001
Epoch 77/499
----------
train Loss: 0.00000833. Learning rate = 0.001
val Loss: 0.00027610. Learning rate = 0.001
Epoch 78/499
----------
train Loss: 0.00001091. Learning rate = 0.001
val Loss: 0.00027813. Learning rate = 0.001
Epoch 79/499
----------
train Loss: 0.00000866. Learning rate = 0.001
val Loss: 0.00027926. Learning rate = 0.001
Epoch 80/499
----------
train Loss: 0.00000934. Learning rate = 0.001
val Loss: 0.00027789. Learning rate = 0.001
Epoch 81/499
----------
train Loss: 0.00000850. Learning rate = 0.001
val Loss: 0.00027688. Learning rate = 0.001
Epoch 82/499
----------
train Loss: 0.000009

val Loss: 0.00027723. Learning rate = 1.0000000000000002e-07
Epoch 142/499
----------
train Loss: 0.00000768. Learning rate = 1.0000000000000002e-07
val Loss: 0.00027652. Learning rate = 1.0000000000000002e-07
Epoch 143/499
----------
train Loss: 0.00000710. Learning rate = 1.0000000000000002e-07
val Loss: 0.00027777. Learning rate = 1.0000000000000002e-07
Epoch 144/499
----------
train Loss: 0.00001682. Learning rate = 1.0000000000000002e-07
val Loss: 0.00027836. Learning rate = 1.0000000000000002e-07
Epoch 145/499
----------
train Loss: 0.00001349. Learning rate = 1.0000000000000002e-07
val Loss: 0.00027861. Learning rate = 1.0000000000000002e-07
Epoch 146/499
----------
train Loss: 0.00000735. Learning rate = 1.0000000000000002e-07
val Loss: 0.00027809. Learning rate = 1.0000000000000002e-07
Epoch 147/499
----------
train Loss: 0.00000777. Learning rate = 1.0000000000000002e-07
val Loss: 0.00027832. Learning rate = 1.0000000000000002e-07
Epoch 148/499
----------
train Loss: 0.000009

train Loss: 0.00000887. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027864. Learning rate = 1.0000000000000004e-08
Epoch 198/499
----------
train Loss: 0.00000915. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027814. Learning rate = 1.0000000000000004e-08
Epoch 199/499
----------
train Loss: 0.00000996. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027895. Learning rate = 1.0000000000000004e-08
Epoch 200/499
----------
train Loss: 0.00000856. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027889. Learning rate = 1.0000000000000004e-08
Epoch 201/499
----------
train Loss: 0.00000779. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027937. Learning rate = 1.0000000000000004e-08
Epoch 202/499
----------
train Loss: 0.00000688. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027834. Learning rate = 1.0000000000000004e-08
Epoch 203/499
----------
train Loss: 0.00000649. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027943. Learning rate = 1.000

train Loss: 0.00000664. Learning rate = 1.0000000000000004e-08
val Loss: 0.00028104. Learning rate = 1.0000000000000004e-08
Epoch 253/499
----------
train Loss: 0.00000844. Learning rate = 1.0000000000000004e-08
val Loss: 0.00028023. Learning rate = 1.0000000000000004e-08
Epoch 254/499
----------
train Loss: 0.00000835. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027843. Learning rate = 1.0000000000000004e-08
Epoch 255/499
----------
train Loss: 0.00000886. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027894. Learning rate = 1.0000000000000004e-08
Epoch 256/499
----------
train Loss: 0.00000932. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027905. Learning rate = 1.0000000000000004e-08
Epoch 257/499
----------
train Loss: 0.00000821. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027906. Learning rate = 1.0000000000000004e-08
Epoch 258/499
----------
train Loss: 0.00000668. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027878. Learning rate = 1.000

val Loss: 0.00027880. Learning rate = 1.0000000000000004e-08
Epoch 308/499
----------
train Loss: 0.00000787. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027712. Learning rate = 1.0000000000000004e-08
Epoch 309/499
----------
train Loss: 0.00000742. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027702. Learning rate = 1.0000000000000004e-08
Epoch 310/499
----------
train Loss: 0.00000664. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027656. Learning rate = 1.0000000000000004e-08
Epoch 311/499
----------
train Loss: 0.00001007. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027746. Learning rate = 1.0000000000000004e-08
Epoch 312/499
----------
train Loss: 0.00000809. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027749. Learning rate = 1.0000000000000004e-08
Epoch 313/499
----------
train Loss: 0.00000972. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027948. Learning rate = 1.0000000000000004e-08
Epoch 314/499
----------
train Loss: 0.000008

val Loss: 0.00027694. Learning rate = 1.0000000000000004e-08
Epoch 364/499
----------
train Loss: 0.00001131. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027754. Learning rate = 1.0000000000000004e-08
Epoch 365/499
----------
train Loss: 0.00001127. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027782. Learning rate = 1.0000000000000004e-08
Epoch 366/499
----------
train Loss: 0.00000902. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027807. Learning rate = 1.0000000000000004e-08
Epoch 367/499
----------
train Loss: 0.00000881. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027784. Learning rate = 1.0000000000000004e-08
Epoch 368/499
----------
train Loss: 0.00000986. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027717. Learning rate = 1.0000000000000004e-08
Epoch 369/499
----------
train Loss: 0.00000857. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027707. Learning rate = 1.0000000000000004e-08
Epoch 370/499
----------
train Loss: 0.000006

val Loss: 0.00027907. Learning rate = 1.0000000000000004e-08
Epoch 419/499
----------
train Loss: 0.00000988. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027825. Learning rate = 1.0000000000000004e-08
Epoch 420/499
----------
train Loss: 0.00000672. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027860. Learning rate = 1.0000000000000004e-08
Epoch 421/499
----------
train Loss: 0.00000764. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027745. Learning rate = 1.0000000000000004e-08
Epoch 422/499
----------
train Loss: 0.00000893. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027788. Learning rate = 1.0000000000000004e-08
Epoch 423/499
----------
train Loss: 0.00000715. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027827. Learning rate = 1.0000000000000004e-08
Epoch 424/499
----------
train Loss: 0.00000883. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027809. Learning rate = 1.0000000000000004e-08
Epoch 425/499
----------
train Loss: 0.000007

val Loss: 0.00027696. Learning rate = 1.0000000000000004e-08
Epoch 474/499
----------
train Loss: 0.00000773. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027685. Learning rate = 1.0000000000000004e-08
Epoch 475/499
----------
train Loss: 0.00000671. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027753. Learning rate = 1.0000000000000004e-08
Epoch 476/499
----------
train Loss: 0.00000621. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027893. Learning rate = 1.0000000000000004e-08
Epoch 477/499
----------
train Loss: 0.00001266. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027962. Learning rate = 1.0000000000000004e-08
Epoch 478/499
----------
train Loss: 0.00000774. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027958. Learning rate = 1.0000000000000004e-08
Epoch 479/499
----------
train Loss: 0.00000637. Learning rate = 1.0000000000000004e-08
val Loss: 0.00027796. Learning rate = 1.0000000000000004e-08
Epoch 480/499
----------
train Loss: 0.000007

In [31]:
list(model.children())[:2]

[Sequential(
   (0): Sequential(
     (0): Linear(in_features=5116, out_features=2048, bias=True)
     (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (2): Dropout(p=0.3, inplace=False)
   )
   (1): Sequential(
     (0): Linear(in_features=2048, out_features=1024, bias=True)
     (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (2): Dropout(p=0.3, inplace=False)
   )
 ), Linear(in_features=1024, out_features=512, bias=True)]

In [32]:
model(X_testTensor)

tensor([[0.7245],
        [0.7789],
        [0.5275],
        [0.7393],
        [0.4920],
        [0.7174],
        [0.6510],
        [0.5865],
        [0.6571],
        [0.6469],
        [0.6408],
        [0.7763],
        [0.6650],
        [0.5562],
        [0.6110],
        [0.7407],
        [0.7636],
        [0.6895],
        [0.7432],
        [0.7756],
        [0.7017],
        [0.5715],
        [0.7456],
        [0.6659],
        [0.6268],
        [0.5246],
        [0.6529],
        [0.7113],
        [0.7618],
        [0.6776],
        [0.6481],
        [0.6101],
        [0.5282],
        [0.7326],
        [0.6451],
        [0.7763],
        [0.7472],
        [0.7428],
        [0.5123],
        [0.8070],
        [0.7397],
        [0.6445],
        [0.6983],
        [0.7747],
        [0.6986],
        [0.7158],
        [0.7268],
        [0.6596],
        [0.7813],
        [0.6869],
        [0.5452],
        [0.6282],
        [0.7645],
        [0.5240],
        [0.5814],
        [0

In [33]:
Y_test

array([[0.88719958],
       [0.80444728],
       [0.55559758],
       [0.91511364],
       [0.50483928],
       [0.7181081 ],
       [0.        ],
       [0.51278461],
       [0.87067471],
       [0.67340947],
       [0.7087871 ],
       [0.80600548],
       [0.82677213],
       [0.37830483],
       [0.79901803],
       [0.91317906],
       [0.74018454],
       [0.7883496 ],
       [0.83080878],
       [0.96176943],
       [0.77377249],
       [0.36045135],
       [0.86045606],
       [0.83492259],
       [0.47074886],
       [0.75372096],
       [0.62928859],
       [0.91529806],
       [0.86323937],
       [0.70538653],
       [0.77163467],
       [0.8388388 ],
       [0.57891795],
       [0.90341582],
       [0.65698058],
       [0.90152264],
       [0.87149522],
       [0.70875888],
       [0.18918441],
       [0.77568825],
       [0.73458215],
       [0.97926911],
       [0.57022927],
       [0.84598245],
       [0.87470572],
       [0.44672655],
       [0.89297698],
       [0.617

In [34]:
from sklearn.metrics import mean_squared_error

In [35]:
dl_result = model(X_testTensor).detach().cpu().numpy()

In [36]:
r2_score(dl_result,Y_test)

-3.8283520686855512

In [37]:
pearsonr(dl_result.flatten(),Y_test.flatten())

(0.502648208132648, 8.733187765846782e-11)

In [38]:
mean_squared_error(dl_result,Y_test)

0.03129790438605715